<a href="https://colab.research.google.com/github/ricardocarreras/Bootcamp_DataScience_student/blob/master/C_Klasse_scraping_part_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar 21 11:27:06 2021
@author: Fabian Schlueter
Thanks to Christopher Buhtz from statisquo.de
His article serves as foundation for this crawler:
https://statisquo.de/2020/01/16/autoscout24-mining-webscraping-mit-python/
"""

# Import packages
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re

In [ ]:
# Paths
filename = 'Mercedes_Benz'   # e.g. name of the car brand or type you search for
path = r'/content/' + filename + '.xlsx'   # select path for saving data

# URL of first result oage of search
# Go to https://www.autoscout24.de and enter your search terms. Select order by age descending.
start_page = 'https://www.autoscout24.de/lst/mercedes-benz/?sort=standard&desc=0&ustate=N%2CU&size=20&page=1&cy=D&atype=C&fc=0&qry=&'

# Existing data of previous search?
# New results will be appended.
try:
    df = pd.read_excel(path)
except FileNotFoundError:
    df = pd.DataFrame()

In [ ]:
# Loop over 20 result pages
for i in range(1,21):
    # Read single page
    print('Reading SRP ' + str(i) + '.')
    # Split up url of start page to fill in current value of i.
    response = requests.get(start_page.split('page=1')[0] + 'page=' + str(i) + start_page.split('page=1')[1])
    html = response.text
    
    doc = BeautifulSoup(html, 'html.parser')
    
    # Get urls of all results on current page.
    offer_list = []
    for paragraph in doc.find_all('a'):
        # Only interested in actual offers (angebote), not in leasing nor recommendation
        if r'/angebote/' in str(paragraph.get('href')) and r'/leasing/' not in str(paragraph.get('href')) and r'/recommendation/' not in str(paragraph.get('href')):
            offer_list.append(paragraph.get('href'))
    
    # Drop urls that were already crawled. These are in df["url"], if df exists from preivous search.
    offer_list_unreduced = offer_list # Just for checking
    try:
        offer_list = [item for item in offer_list if 'https://www.autoscout24.de' + item not in list(df["url"])]
    except:
        print("First results for this search.")

    # Loop over offers.
    for item in offer_list:
        try:
            url = 'https://www.autoscout24.de' + item
            response = requests.get(url)
            html = response.text
            
            doc = BeautifulSoup(html, 'html.parser')
            
            # Empty dictionary for saving car's main features
            car_dict = {}
            
            # Names of main features are within dt tags of html. Their value is always in the following dd tag.
            for key, value in zip(doc.find_all('dt'), doc.find_all('dd')): # Combine every dt tag with the following dd tag by zip.
                car_dict[key.text.replace("\n", "")] = value.text.replace("\n", "") # Save in dict.
            
            # Following features must be identified separateley.
            
            # professional seller?
            car_dict['haendler'] = doc.find("div", attrs={"class":"cldt-vendor-contact-box",
                                                          "data-vendor-type":"dealer"}) != None
            
            # private seller?
            car_dict['privat'] = doc.find("div", attrs={"class":"cldt-vendor-contact-box",
                                                          "data-vendor-type":"privateseller"}) != None
            # city of sale incl. zip-code
            car_dict['ort'] = doc.find("div", attrs={"class":"sc-grid-col-12",
                                                          "data-item-name":"vendor-contact-city"}).text
            # driven miles
            car_dict['miles'] = html.split('"stmil" : ')[1].replace("\n", '').split(',')[0].strip()
            
            # price
            car_dict['price'] = "".join(re.findall(r'[0-9]+',doc.find("div",attrs={"class":"cldt-price"}).text))
            
            # save url and time of program's execution
            car_dict['url'] = url
            car_dict['date'] = datetime.now().strftime("%Y-%m-%d")
            car_dict['time'] = datetime.now().strftime("%H-%M-%S")
            
            # add several features that have no value. These either exist in the current car or not (e.g. air-con, radio, leather seatings, etc.)
            for j in doc.find_all('div', attrs={"class":"cldt-equipment-block sc-grid-col-3 sc-grid-col-m-4 sc-grid-col-s-12 sc-pull-left"}):
                for span in j.find_all('span'):
                    car_dict[span.text] = 1 # assign value of 1 if feature exists.

            # append data of current car to dataframe
            car_car_dict = {}
            car_car_dict['URL'] = car_dict
            df_append = pd.DataFrame(car_car_dict).T
            df=df.append(df_append)   
            
        except Exception as e:
            print(str(e))

    print('Appended data from SRP ' + str(i) + '.')

Reading SRP 1.
Appended data from SRP 1.
Reading SRP 2.
First results for this search.
Appended data from SRP 2.
Reading SRP 3.
Appended data from SRP 3.
Reading SRP 4.
Appended data from SRP 4.
Reading SRP 5.
Appended data from SRP 5.
Reading SRP 6.
Appended data from SRP 6.
Reading SRP 7.
Appended data from SRP 7.
Reading SRP 8.
Appended data from SRP 8.
Reading SRP 9.
Appended data from SRP 9.
Reading SRP 10.
Appended data from SRP 10.
Reading SRP 11.
Appended data from SRP 11.
Reading SRP 12.
Appended data from SRP 12.
Reading SRP 13.
Appended data from SRP 13.
Reading SRP 14.
Appended data from SRP 14.
Reading SRP 15.
Appended data from SRP 15.
Reading SRP 16.
Appended data from SRP 16.
Reading SRP 17.
Appended data from SRP 17.
Reading SRP 18.
Appended data from SRP 18.
Reading SRP 19.
Appended data from SRP 19.
Reading SRP 20.
Appended data from SRP 20.


In [ ]:
df.shape

(260, 183)

In [ ]:
pd.set_option('display.max_columns',df.shape[1])

In [ ]:
df

,ABS,Angebotsnummer,Anzahl Türen,Armlehne,Außenfarbe,Beifahrerairbag,Bordcomputer,"CO2-EmissionenWeitere Informationen zum offiziellen Kraftstoffverbrauch und den offiziellen spezifischen CO2-Emissionen neuer Personenkraftwagen können dem ""Leitfaden über den Kraftstoffverbrauch, die CO2-Emissionen und den Stromverbrauch neuer Personenkraftwagen"" entnommen werden, der an allen Verkaufsstellen und bei der Deutschen Automobil Treuhand GmbH unter www.dat.de unentgeltlich erhältlich ist.",Dachreling,E10-geeignet,ESP,Elektr. Fensterheber,Elektrische Seitenspiegel,Erstzulassung,Fahrerairbag,Farbe laut Hersteller,Feinstaubplakette,Geschwindigkeits-begrenzungsanlage,Getriebeart,Getönte Scheiben,Gänge,Hubraum,Innenausstattung,Isofix,Karosserieform,Klimaanlage,Klimaautomatik,Kopfairbag,Kraftstoff,"KraftstoffverbrauchWeitere Informationen zum offiziellen Kraftstoffverbrauch und den offiziellen spezifischen CO2-Emissionen neuer Personenkraftwagen können dem ""Leitfaden über den Kraftstoffverbrauch, die CO2-Emissionen und den Stromverbrauch neuer Personenkraftwagen"" entnommen werden, der an allen Verkaufsstellen und bei der Deutschen Automobil Treuhand GmbH unter www.dat.de unentgeltlich erhältlich ist.",Lackierung,Länderversion,MP3,Marke,Modell,Nebelscheinwerfer,Nichtraucherfahrzeug,Notrad,Radio,Schadstoffklasse,Scheckheftgepflegt,Schlüsselnummer,Seitenairbag,Servolenkung,Sitzplätze,Stahlfelgen,Tagfahrlicht,Tempomat,Traktionskontrolle,Wegfahrsperre,Zentralverriegelung,Zustand,date,haendler,miles,ort,price,privat,time,url,Alufelgen,Fahrzeughalter,Multifunktionslenkrad,Scheinwerferreinigung,Xenonscheinwerfer,CD,HU Prüfung,HU/AU neu,Sitzheizung,Antriebsart,Bluetooth,Elektrische Sitze,Freisprecheinrichtung,Katalysator,Navigationssystem,Regensensor,Schiebedach,Soundsystem,Zylinder,Alarmanlage,Ausstattung,Bilder,Einparkhilfe Sensoren hinten,Einparkhilfe Sensoren vorne,Elektrische Heckklappe,Fahrzeugbeschreibung,Getriebe,Kilometerstand,Kraftstoffverbr.*,Lederlenkrad,Leergewicht,Leistung,Letzter Kundendienst,Lichtsensor,"Marke, Modell",Modellvarianten,Region,Reserverad,Serviceleistungen,Sommerreifen,Spezielle Umbauten,Sportsitze,USB,Verkäufer,Windschott(für Cabrio),Winterreifen,Zentralverriegelung mit Funkfernbedienung,Allrad,Anhängerkupplung,Berganfahrassistent,Reifendruckkontrollsystem,teilb. Rücksitzbank,Kurvenlicht,Sportpaket,Lordosenstütze,Allwetterreifen,Beheizbare Frontscheibe,Einparkhilfe selbstlenkendes System,Gepäckraumabtrennung,Innenspiegel automatisch abblendend,Raucherpaket,Winterpaket,Pannenkit,2-Zonen-Klimaautomatik,Bi-Xenon Scheinwerfer,Panoramadach,Sportfahrwerk,Umklappbarer Beifahrersitz,Airbag hinten,Garantie,Skisack,Totwinkel-Assistent,Schaltwippen,Einparkhilfe,Luftfederung,Schiebetür rechts,Abstandswarner,LED-Tagfahrlicht,Müdigkeitswarnsystem,Notbremsassistent,Sprachsteuerung,Start/Stop-Automatik,Ambientebeleuchtung,Musikstreaming integriert,3-Zonen-Klimaautomatik,Standheizung,Abstandstempomat,Beheizbares Lenkrad,Fernlichtassistent,Sitzbelüftung,Einparkhilfe Kamera,Spurhalteassistent,Nutzlast,Schiebetür,Trennwand,Andere Energieträger,LED-Scheinwerfer,Verkehrszeichenerkennung,Voll-LED Scheinwerfer,Elektronische Parkbremse,Schlüssellose Zentralverriegelung,Notrufsystem,Touchscreen,Produktionsjahr,zul. Gesamtgewicht,DAB-Radio,Spoiler,Schiebetür links,Taxi oder Mietwagen,Blendfreies Fernlicht,W-Lan / Wifi Hotspot,"Elektr. Sitzeinstellung, hinten",Nachtsicht-Assistent,Android Auto,Apple CarPlay,TV,Massagesitze,Tuning,Biodieselumrüstung,Anzahl Achsen,Doppelkabine,360° Kamera,Volldigitales Kombiinstrument
URL,1,1F737311,5,1,Schwarz,1,1,224 g/km (komb),1,1,1,1,1,2005,1,Obsidianschwarz Metallic,4 (Grün),1,Automatik,1,5,1.796 cm³,"Stoff, Schwarz",1,Kombi,1,1,1,Normal/Benzin 91,"9,3 l/100 km (komb)13,1 l/100 km (innerorts)7,...",Metallic,Deutsche Ausführung,1,Mercedes-Benz,C 200,1,,1,1,Euro 4,,0710/567,1,1,5,1,1,1,1,1,1,Gebraucht,2021-08-23,True,184985,94315 Straubing,2989,False,09-10-33,https://www.autoscout24.de/angebote/mercedes-b

In [ ]:
# save dataframe as excel
df.to_excel(path, index=False)